In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import json
 
# Paths to the dataset directories
train_dir = 'train'
valid_dir = 'valid'
test_dir = 'test'
 
# Image dimensions
img_height, img_width = 640, 640
batch_size = 16
 
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
 
valid_datagen = ImageDataGenerator(rescale=1./255)
 
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)
 
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)
 
# Number of classes
num_classes = len(train_generator.class_indices)
 
# Load MobileNetV2 model with pre-trained weights and without top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
 
# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
 
# Define the model
model = Model(inputs=base_model.input, outputs=predictions)
 
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False
 
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
 
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=valid_generator
)
 
# Save the model
model.save('staff_mobilenet_v2_model.h5')
 
# Save class names
class_names = list(train_generator.class_indices.keys())
with open('class_names.json', 'w') as f:
    json.dump(class_names, f)
 
print("Model and class names saved successfully.")

Found 63 images belonging to 3 classes.
Found 18 images belonging to 3 classes.


C:\Users\Bryan Tan\AppData\Local\Temp\ipykernel_13748\3460206690.py:46: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/10


c:\Users\Bryan Tan\anacoda3\envs\MLDP_AY23_BryanTan\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 19s 4s/step - accuracy: 0.5002 - loss: 0.9565 - val_accuracy: 0.7222 - val_loss: 0.7270
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - accuracy: 0.9633 - loss: 0.6511 - val_accuracy: 1.0000 - val_loss: 0.4489
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 0.9600 - loss: 0.4764 - val_accuracy: 1.0000 - val_loss: 0.2771
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 1.0000 - loss: 0.2777 - val_accuracy: 1.0000 - val_loss: 0.1940
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 1.0000 - loss: 0.1972 - val_accuracy: 1.0000 - val_loss: 0.1517
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 1.0000 - loss: 0.1259 - val_accuracy: 1.0000 - val_loss: 0.1043
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 1.0000 - loss: 0.0993 - val_accuracy: 1.0000 - val_loss: 0.0717
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 1.0000 - loss: 0.0657 - val_accuracy: 1.0000 - val_loss: 0.0513
Epoch 9/10
4/4 ━━━━

Model and class names saved successfully.


In [2]:
# check_versions.py
 
import streamlit as st
import numpy as np
from PIL import Image
import tensorflow as tf
 
def check_versions():
    # Streamlit version
    st_version = st.__version__
    print(f"Streamlit version: {st_version}")
 
    # NumPy version
    np_version = np.__version__
    print(f"NumPy version: {np_version}")
 
    # Pillow (PIL) version
    pil_version = Image.__version__
    print(f"Pillow version: {pil_version}")
 
    # TensorFlow version
    tf_version = tf.__version__
    print(f"TensorFlow version: {tf_version}")
 
if __name__ == "__main__":
    check_versions()

Streamlit version: 1.23.1
NumPy version: 1.23.5
Pillow version: 10.4.0
TensorFlow version: 2.17.0
